### Import Modules

In [154]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [155]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### Input Data

In [156]:
Recyling_Products = [12000, 8000, 19000, 21000, 25000, 5000]
Construction_Costs = [12000, 19500, 32000]
Handling_Charges = [6, 2, 8]
Processing_Capacity = [0, 38000, 18100]

Cik = [
    [1, 9, 9, 4, 9],
    [8, 4, 2, 7, 4],
    [8, 3, 6, 6, 6],
    [1, 2, 5, 4, 4],
    [9, 6, 4, 1, 8],
    [6, 3, 2, 2, 4]
]

Ckl = [
    [8, 9, 7, 9],
    [11, 8, 4, 10],
    [9, 3, 5, 4],
    [5, 4, 8, 6],
    [7, 6, 3, 8]
]

Cli = [
    [6, 5, 4, 6, 7, 2],
    [4, 2, 6, 3, 7, 2],
    [2, 3,1, 5, 8, 2], 
    [7, 4, 5, 7, 5, 9]
]


# in dollars

Eijk = [
    [2, 5, 3, 1, 2],  
    [3, 2, 4, 2, 3],
    [1, 2, 2, 3, 1],
    [4, 2, 3, 2, 4],
    [3, 8, 2, 1, 3],
    [2, 3, 9, 11, 8]
]

Eikl = [
    [5, 2, 3, 5],  
    [6, 3, 2, 3],
    [3, 4, 1, 2],
    [1, 2, 7, 8],
    [2, 7, 2, 6]
]

Elji = [
    [5, 3, 8, 3, 2, 1],  
    [2, 6, 2, 6, 5, 2],
    [4, 8, 3, 2, 4, 3],
    [6, 2, 2, 3, 6, 7]
]


### Creating Model and Declaring Variables

In [157]:

model = gp.Model("closed_loop_supply_chain")


Customer_Centres = model.addVars(5, vtype = gp.GRB.BINARY, name = 'Customer_Centres')
Split_Testing_Centres = model.addVars(6, vtype=gp.GRB.BINARY, name="Split_Testing_Centres")
Remanufacturing_Plants = model.addVars(4, vtype=GRB.BINARY, name="Remanufacturing_Plants")


quantity_ijk = model.addVars(6, 5, name="quantity_ijk")
quantity_ikl = model.addVars(5, 4, name="quantity_ikl")
quantity_lji = model.addVars(4, 6, name="quantity_lji")


EnvironmentCost_ijk = model.addVars(6, 5, name="EnvironmentCost_ijk")
EnvironmentCost_ikl = model.addVars(5, 4, name="EnvironmentCost_ikl")
EnvironmentCost_lji = model.addVars(4, 6, name="EnvironmentCost_lji")


### Objective Function

In [158]:
Construction_Cost_Centres = gp.quicksum(Construction_Costs[0] * Customer_Centres[i] for i in range (5))
Construction_Cost_Split = gp.quicksum(Construction_Costs[1] * Split_Testing_Centres[j] for j in range(6))
Construction_Cost_Remanufacturing = gp.quicksum(Construction_Costs[2] * Remanufacturing_Plants[l] for l in range(4))


Handling_Cost_1 = gp.quicksum(Handling_Charges[0] * quantity_ijk[i, j] for i in range(6) for j in range(5))
Handling_Cost_2 = gp.quicksum(Handling_Charges[1] * quantity_ikl[j, k] for j in range(5) for k in range(4))
Handling_Cost_3 = gp.quicksum(Handling_Charges[2] * quantity_lji[k, l] for k in range(4) for l in range(6))


Processing_Cost_1 = gp.quicksum(Cik[i][j] * quantity_ijk[i, j] for i in range(6) for j in range(5))
Processing_Cost_2 = gp.quicksum(Ckl[j][k] * quantity_ikl[j, k] for j in range(5) for k in range(4))
Processing_Cost_3 = gp.quicksum(Cli[k][i] * quantity_lji[k, i] for k in range(4) for i in range(6))

Env_impact_cost_1 = gp.quicksum(Eijk[i][j] * quantity_ijk[i, j] for i in range(6) for j in range(5))
Env_impact_cost_2 = gp.quicksum(Eikl[j][k] * quantity_ikl[j, k] for j in range(5) for k in range(4))
Env_impact_cost_3 = gp.quicksum(Elji[k][i] * quantity_lji[k, i] for k in range(4) for i in range(6))

model.setObjective(Construction_Cost_Split + Construction_Cost_Remanufacturing + 
               Handling_Cost_1 + Processing_Cost_1 + Processing_Cost_2 + Processing_Cost_3 + Construction_Cost_Centres + 
               Handling_Cost_2 + Handling_Cost_3 +  Env_impact_cost_1 + Env_impact_cost_2 + Env_impact_cost_3
               , GRB.MINIMIZE)


### Constraints and Optimizing the Model

In [159]:

for j in range(5):
    model.addConstr(gp.quicksum(quantity_ijk[i, j] for i in range(5)) <= Processing_Capacity[1] * Split_Testing_Centres[j])

for k in range(4):
    model.addConstr(gp.quicksum(quantity_ikl[j, k] for j in range(4)) <= Processing_Capacity[2] * Remanufacturing_Plants[k])

for i in range(6):
    model.addConstr(gp.quicksum(quantity_ijk[i, j] for j in range(3)) == Recyling_Products[i])


for k in range(4):
    model.addConstr(gp.quicksum(quantity_ikl[j, k] for j in range(4)) == gp.quicksum(quantity_lji[k, i] for i in range(5)))
    

for i in range(4):
    model.addConstr(Customer_Centres[i] <=1)

for j in range(5):
    model.addConstr(Split_Testing_Centres[j] <= 1)
    
for k in range(4):
    model.addConstr(Remanufacturing_Plants[k] <= 1)

    

for i in range(6):
    for j in range(5):
        model.addConstr(EnvironmentCost_ijk[i, j] >= 1)
 
        
for j in range(5):
    for k in range(4):
        model.addConstr(EnvironmentCost_ikl[j,k] >=1)
     
        

for l in range(4):
    for i in range(6):
        model.addConstr(EnvironmentCost_lji[k, i] >=1)
        
        

for i in range(6):
    for j in range(5):
        model.addConstr(quantity_ijk[i, j] >= 1)
 
        
for j in range(5):
    for k in range(4):
        model.addConstr(quantity_ikl[j,k] >=1)
     
        

for l in range(4):
    for i in range(6):
        model.addConstr(quantity_lji[k, i] >=1)        
        
        
        
        
        
    
    
model.addConstr(gp.quicksum(Split_Testing_Centres[j] for j in range(3)) >= 1)
model.addConstr(gp.quicksum(Remanufacturing_Plants[k] for k in range(4)) >= 1)
model.addConstr(gp.quicksum(Customer_Centres[i] for i in range(4)) >= 1)

model.optimize()



Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-9300H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 183 rows, 163 columns and 276 nonzeros
Model fingerprint: 0x2456c034
Variable types: 148 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [7e+00, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]
Found heuristic solution: objective 1550314.0000
Presolve removed 175 rows and 148 columns
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros
Found heuristic solution: objective 1525199.0000
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 1.223264e+06, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntIn